In [1]:
x = 1 + 1.2j

In [2]:
import numpy as np

In [3]:
np.abs(x)

1.5620499351813308

In [4]:
foo = np.array([1 + 1.2j, 3 + 4j, 5 + 6j])
np.abs(foo)

array([1.56204994, 5.        , 7.81024968])

In [28]:
import networkit as nk
import networkx as nx
import scipy
import numpy as np
Iweighting=0.5
sparse_evals=10

g = nk.generators.ErdosRenyiGenerator(1000, 0.1).generate()

In [26]:
nk.overview(g)

Network Properties:
nodes, edges			1000, 49854
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.099808
clustering coefficient		0.099740
min/max/avg degree		69, 129, 99.708000
degree assortativity		-0.005520
number of connected components	1
size of largest component	1000 (100.00 %)


In [47]:
gnx = nk.nxadapter.nk2nx(g)

# A = nx.linalg.adjacency_matrix(gnx).todense()
A = nx.linalg.adjacency_matrix(gnx)

D = np.array([x[1] for x in (gnx.degree)])
D_h = D**(0.5)
D_hi = D**(-0.5)

n = A.shape[0]



M = (A.T).multiply(1/D).T

# M = (1-Iweighting)* M + Iweighting * np.eye(M.shape[0])
# M = (1 - Iweighting) * M + Iweighting * (np.ones((n, n))/n if eye_or_ones == "ones" else np.eye(n))
# sparse array version
# M = (1 - Iweighting) * M + scipy.sparse.eye(n) * Iweighting

# S_ij = W_ij / sqrt(d_i d_j) = sqrt(d_i) M_ij / sqrt(d_j)
# S = np.diag(D_h) @ M @ np.diag(D_hi)
# sparse array version
S = scipy.sparse.diags(D_h) @ M @ scipy.sparse.diags(D_hi)

# w, V = np.linalg.eigh(S)
# sparse array version
# w, V = scipy.sparse.linalg.eigsh(S, k=sparse_evals, which="LM")
w, V = scipy.linalg.eig(S.todense())

# S = V @ np.diag(w) @ V.T
# M = D^{-1/2} @ S @ D^{1/2}
# = D^{-1/2} @ V @ np.diag(w) @ V.T @ D^{1/2}
# = Phi @ np.diag(w) @ Psi.T
# Phi = D^{-1/2} @ V
# Psi = D^{1/2} @ V
# Phi = np.diag(D_hi) @ V
# Psi = np.diag(D_h) @ V
# sparse array version
Phi = scipy.sparse.diags(D_hi) @ V
Psi = scipy.sparse.diags(D_h) @ V


# Empirically this gamma seems to work well.
# It discourages taking edges to popular nhbs.
gamma = 0.9
M_tilde1 = scipy.sparse.diags(1 / D) @ A @ scipy.sparse.diags(D ** (-gamma))
D1 = scipy.sparse.diags(np.array(1 / M_tilde1.sum(axis=-1)).squeeze())
M_tilde = D1 @ M_tilde1
# M_tilde = scipy.sparse.diags(np.array(1 / M_tilde.sum(axis=-1)).squeeze()) @ M_tilde
# M_tilde = (1 - Iweighting) * M_tilde + Iweighting * np.eye(M_tilde.shape[0])
# a, B = scipy.sparse.linalg.eigs(M_tilde, k=sparse_evals, which="LR")
a,B = scipy.linalg.eig(M_tilde.todense())
# a, B = np.real(a), np.real(B)

idx = np.argsort(a)[::-1]
a = a[idx]
B = B[:, idx]

def diff_map(i, t):
    return B[i, 1:] * a[1:] ** t

In [61]:
B_inv = np.linalg.inv(B)
temp = M_tilde - B @ np.diag(a) @ B_inv
temp2 = M_tilde - D1 @ Phi @ np.diag(w) @ Psi.T @ np.diag(D**(-gamma))
temp3 = M - Phi @ np.diag(w) @ Psi.T

In [63]:
a

array([ 1.00000000e+00+0.j,  1.90184874e-01+0.j,  1.87197041e-01+0.j,
        1.86798505e-01+0.j,  1.85532709e-01+0.j,  1.84851293e-01+0.j,
        1.84026290e-01+0.j,  1.82106505e-01+0.j,  1.80947093e-01+0.j,
        1.80007196e-01+0.j,  1.78727760e-01+0.j,  1.78070346e-01+0.j,
        1.77256936e-01+0.j,  1.76875450e-01+0.j,  1.76389445e-01+0.j,
        1.76012815e-01+0.j,  1.75596960e-01+0.j,  1.74078039e-01+0.j,
        1.73375288e-01+0.j,  1.72172366e-01+0.j,  1.70872449e-01+0.j,
        1.70424229e-01+0.j,  1.69891366e-01+0.j,  1.69715020e-01+0.j,
        1.69281410e-01+0.j,  1.68135201e-01+0.j,  1.67624514e-01+0.j,
        1.66352405e-01+0.j,  1.66150875e-01+0.j,  1.65489115e-01+0.j,
        1.65113746e-01+0.j,  1.65046159e-01+0.j,  1.63833702e-01+0.j,
        1.63237945e-01+0.j,  1.62885682e-01+0.j,  1.62564156e-01+0.j,
        1.60890889e-01+0.j,  1.60526370e-01+0.j,  1.59619157e-01+0.j,
        1.59165098e-01+0.j,  1.58271628e-01+0.j,  1.57890743e-01+0.j,
        1.57100706e-

In [66]:
foo = M_tilde @ M_tilde.T - (M_tilde.T @ M_tilde)
foo.todense()

matrix([[-2.12491898e-04, -6.02431184e-06, -5.41462043e-06, ...,
         -4.39003192e-05, -2.87817280e-05, -1.08204422e-05],
        [-6.02431184e-06,  1.87890043e-04,  1.14872684e-05, ...,
         -9.85517250e-06, -7.23301913e-06,  2.53390292e-05],
        [-5.41462043e-06,  1.14872684e-05,  3.53774851e-06, ...,
         -1.49249842e-05, -1.12515287e-05,  1.18490020e-05],
        ...,
        [-4.39003192e-05, -9.85517250e-06, -1.49249842e-05, ...,
         -4.09642505e-04, -3.96749184e-05, -3.36316522e-05],
        [-2.87817280e-05, -7.23301913e-06, -1.12515287e-05, ...,
         -3.96749184e-05, -2.87736821e-04, -8.32092153e-07],
        [-1.08204422e-05,  2.53390292e-05,  1.18490020e-05, ...,
         -3.36316522e-05, -8.32092153e-07,  5.54109686e-05]])

0.001233990106009773

In [55]:
M_tilde1 = scipy.sparse.diags(1 / D) @ A @ scipy.sparse.diags(D ** (-gamma))
M_tilde2 = scipy.sparse.diags(1 / D) @ A @ np.diag(D ** (-gamma))

In [59]:
M_tilde1 - (M @ np.diag(D**(-gamma)))

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [60]:
D1 @ M_tilde1 - (D1 @ M @ np.diag(D**(-gamma)))

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])